In [1]:
%load_ext autoreload
import datetime
# import ete3
import itertools
import json
import logging
import math
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import typing
# import re
import xml.etree.ElementTree as ET
# import xml.dom.minidom
# import xmlschema

In [2]:
%autoreload
from maxes.xes_loader2 import XesLoader, XesLog
# from maxes.xes_file import XesFile
# from maxes.analyze_xes import AnalyzeXes
# import maxes.analyze_sequence
# import maxes.serialization.serialize
# import maxes.graphs
import maxes.notebooks.utils
import maxes.utils
from maxes.analyze_xes import AnalyzeXes

In [ ]:
maxes.notebooks.utils.init_notebook() # RUN ONLY ONCE

In [ ]:
import maxes.data.load_files

file_path = maxes.data.load_files.get_path_photo_copier()
file_path

In [5]:
import maxes.notebooks.utils

log = maxes.notebooks.utils.load_xes(file_path)
log.update_df()
len(log.traces)

100

In [6]:
from maxes.generators.xes_generator.xes_generator3 import XesGenerator3

# Initialize generator
xes_generator = XesGenerator3(attributes_models=[], traces_count=1)
xes_generator.fit(log)

In [11]:
xes_generator.traces_count = 100

generated_log = xes_generator.generate()
len(generated_log.traces)

100

In [12]:
for trace_index, trace in enumerate(generated_log.traces):
    trace.attributes["concept:name"] = str(trace_index)

generated_log.update_df(with_trace_attributes=True)
generated_log_pm4py = generated_log.df
generated_log_pm4py

,concept:name,lifecycle:transition,time:timestamp,case:concept:name
0,Job,start,1970-01-03 20:34:00.771391+00:00,0
1,Remote Print,complete,1970-01-04 04:49:00.513882+00:00,0
2,Read Print Options,complete,1970-01-04 05:01:59.631726+00:00,0
3,Rasterization,start,1970-01-04 05:12:59.563689+00:00,0
4,Interpretation,start,1970-01-04 05:32:59.719534+00:00,0
...,...,...,...,...
265,Pressure Roller Spin Stop,complete,1970-01-08 15:04:06.805457+00:00,99
266,Fusing,complete,1970-01-08 15:10:06.602493+00:00,99
267,Wipe Toner on Drum,complete,1970-01-08 15:19:08.360796+00:00,99
268,Erase Charge on Drum,complete,1970-01-08 15:31:07.162676+00:00,99


In [9]:
import pm4py

In [12]:
log_pm4py = pm4py.read_xes(file_path, show_progress_bar=False)
log_pm4py

,org:resource,time:timestamp,concept:name,lifecycle:transition,case:concept:name,case:description,case:Class
0,PRN1,1970-01-01 01:00:00+00:00,Job,start,1,Simulated process instance,Print
1,PRN1,1970-01-01 01:15:00+00:00,Remote Print,complete,1,Simulated process instance,Print
2,PRN1,1970-01-01 01:26:00+00:00,Read Print Options,complete,1,Simulated process instance,Print
3,PRN1,1970-01-01 01:38:00+00:00,Rasterization,start,1,Simulated process instance,Print
4,PRN1,1970-01-01 01:51:00+00:00,Interpretation,start,1,Simulated process instance,Print
...,...,...,...,...,...,...,...
40990,PRN1,1970-12-18 19:46:00+00:00,Compression,complete,99,Simulated process instance,Copy/Scan
40991,PRN1,1970-12-18 19:53:00+00:00,Store Image,complete,99,Simulated process instance,Copy/Scan
40992,PRN1,1970-12-18 20:11:00+00:00,Transfer Image,complete,99,Simulated process instance,Copy/Scan
40993,PRN1,1970-12-18 20:18:00+00:00,Send SMTP,complete,99,Simulated process instance,Copy/Scan


In [ ]:
net, im, fm = pm4py.discover_petri_net_inductive(log_pm4py)

In [18]:
from pm4py.algo.evaluation.replay_fitness.variants import token_replay as fitness_tbr
from pm4py.algo.evaluation.generalization.variants import token_based as generalization_tbr
from pm4py.algo.evaluation.precision.variants import etconformance_token as precision_tbr

fitness = fitness_tbr.apply(generated_log_pm4py, net, im, fm, {'show_progress_bar': False})
print(fitness)

generalization = generalization_tbr.apply(generated_log_pm4py, net, im, fm, {'show_progress_bar': False})
print(generalization)

precision = precision_tbr.apply(generated_log_pm4py, net, im, fm, {'show_progress_bar': False})
print(precision)

{'perc_fit_traces': 0.0, 'average_trace_fitness': 0.9657836379403072, 'log_fitness': 0.9765307709858082, 'percentage_of_fitting_traces': 0.0}
0.9164326430028654


KeyboardInterrupt: 

In [19]:
print("Precision: ", pm4py.conformance.precision_token_based_replay(generated_log_pm4py, net, im, fm))


replaying log with TBR, completed traces ::   0%|          | 0/39918 [00:00<?, ?it/s]

Precision:  0.35008873577617705


In [ ]:
print("Precision: ", pm4py.conformance.precision_alignments(generated_log_pm4py, net, im, fm))


In [7]:
xes_generator.traces_count = 10

generated_log = xes_generator.generate()

for trace_index, trace in enumerate(generated_log.traces):
    trace.attributes["concept:name"] = str(trace_index)

generated_log.update_df(with_trace_attributes=True)
generated_log_pm4py = generated_log.df
generated_log_pm4py

,concept:name,lifecycle:transition,time:timestamp,case:concept:name
0,Job,start,1970-01-02 21:24:00.071865+00:00,0
1,Copy/Scan,complete,1970-01-03 14:57:58.265290+00:00,0
2,Collect Copy/Scan Options,complete,1970-01-03 15:06:00.765714+00:00,0
3,Place Doc,complete,1970-01-03 15:31:02.524060+00:00,0
4,Illuminate Document,complete,1970-01-03 15:42:01.696471+00:00,0
...,...,...,...,...
78,Compression,complete,1970-01-08 01:10:08.101412+00:00,9
79,Store Image,complete,1970-01-08 01:18:07.402812+00:00,9
80,Transfer Image,complete,1970-01-08 01:40:07.489295+00:00,9
81,Send FTP,complete,1970-01-08 02:00:07.846040+00:00,9


In [14]:
print("Precision: ", pm4py.conformance.precision_token_based_replay(generated_log_pm4py, net, im, fm))


replaying log with TBR, completed traces ::   0%|          | 0/4503 [00:00<?, ?it/s]

Precision:  0.2728592162554426


In [ ]:
file_path = maxes.data.load_files.get_path_software_data_analytics__alarm_system()
print(file_path)
AnalyzeXes.s_analyze(file_path)

In [ ]:
maxes.data.load_files.get_path_software_data_analytics__junit

In [17]:
import maxes.data.manifest_reader

keys = [
    "alarm_system",
    "book_store_2",
    "book_store_20",
    "sports_news_commentary",
    "design_patterns__observer__pattern1",
    "design_patterns__state__remote_control_software",
    "design_patterns__observer__sensing_alarm_system",
    "design_patterns__observer__short_message_subscribe",
    "design_patterns__state__pattern1",
    "design_patterns__strategy__test_software",
    "jgraphx",
    "jhotdraw",
    "junit",
    "lexi",
    "lexi_transformed",
    "mailing_server__observer_pattern",
]

for i, file_key in enumerate(keys):
    file_path = maxes.data.manifest_reader.get_file_path("software_data_analytics", file_key)
    log_pm4py = pm4py.read_xes(file_path, show_progress_bar=False)
    traces_count = log_pm4py["case:concept:name"].nunique()
    print(f"{i=}, {file_key=}, {traces_count=}")

i=0, file_key='alarm_system', traces_count=5
i=1, file_key='book_store_2', traces_count=2
i=2, file_key='book_store_20', traces_count=20
i=3, file_key='sports_news_commentary', traces_count=10
i=4, file_key='design_patterns__observer__pattern1', traces_count=1
i=5, file_key='design_patterns__state__remote_control_software', traces_count=1
i=6, file_key='design_patterns__observer__sensing_alarm_system', traces_count=3
i=7, file_key='design_patterns__observer__short_message_subscribe', traces_count=5
i=8, file_key='design_patterns__state__pattern1', traces_count=1
i=9, file_key='design_patterns__strategy__test_software', traces_count=1
i=10, file_key='jgraphx', traces_count=1
i=11, file_key='jhotdraw', traces_count=1
i=12, file_key='junit', traces_count=1
i=13, file_key='lexi', traces_count=1
i=14, file_key='lexi_transformed', traces_count=4
i=15, file_key='mailing_server__observer_pattern', traces_count=3


In [31]:
file_path = maxes.data.manifest_reader.get_file_path("software_data_analytics", "alarm_system")
log_pm4py = pm4py.read_xes(file_path)
log_pm4py

parsing log, completed traces ::   0%|          | 0/5 [00:00<?, ?it/s]

,software:classobject,software:class,software:callerclassobject,software:callerclass,software:linenumber,lifecycle:transition,software:callerpackage,software:threadid,software:starttimenano,software:parametertypeset,software:package,concept:name,software:endtimenano,software:callermethod,case:concept:name,software:returnvalue,Nested_Flag,software:parametervalueset
0,1909173090,Console,@execution1,Console,37,start,com.verum.examples.AlarmSystem.Console,1,1780228355339781,,com.verum.examples.AlarmSystem.Console,init(),1780228355339781,main(),execution1,NaN,NaN,NaN
1,1585995280,LogWindow,872627444,Console,110,start,com.verum.examples.AlarmSystem.Console,15,1780229123870087,com.verum.examples.AlarmSystem.Console.Console...,com.verum.examples.AlarmSystem.Console,init(),1780229123870087,init(),execution1,NaN,NaN,NaN
2,0,AlarmSystemComponent,872627444,Console,114,start,com.verum.examples.AlarmSystem.Console,15,1780229125827882,,com.verum.examples.AlarmSystem,getInstance(),1780229740074283,init(),execution1,1574600022,0,
3,1585995280,LogWindow,872627444,Console,306,start,com.verum.examples.AlarmSystem.Console,15,1780229163020426,java.lang.String,com.verum.examples.AlarmSystem.Console,showInfo(),1780229177334824,traceHandler(),execution1,void,NaN,476977873
4,673733348,State,673733348,AlarmSystem,308,start,com.verum.examples,15,1780229406022635,com.verum.examples.AlarmSystem.AlarmSystemImpl...,com.verum.examples.AlarmSystem,init(),1780229406022635,AlarmSystemImplScope$NotActivated(),execution1,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,1634888284,LogWindow,1751432579,Console,306,start,com.verum.examples.AlarmSystem.Console,19,1780414976124994,java.lang.String,com.verum.examples.AlarmSystem.Console,showInfo(),1780414977585856,traceHandler(),execution5,void,NaN,2127570078
931,0,Console,1921707273,Console,237,start,com.verum.examples.AlarmSystem.Console,15,1780414988771545,com.verum.examples.AlarmSystem.Console.Console,com.verum.examples.AlarmSystem.Console,access$3(),1780414989004189,run(),execution5,1906576862,NaN,1751432579
932,1751432579,Console,1921707273,Console,237,start,com.verum.examples.AlarmSystem.Console,15,1780414989127780,java.lang.String,com.verum.examples.AlarmSystem.Console,createImageIcon(),1780414989818867,run(),execution5,320094749,NaN,306384091
933,0,Console,1921707273,Console,238,start,com.verum.examples.AlarmSystem.Console,15,1780414990106678,com.verum.examples.AlarmSystem.Console.Console...,com.verum.examples.AlarmSystem.Console,access$4(),1780414990242244,run(),execution5,void,NaN,"1751432579,0"


In [35]:
import datetime

for index, row in log_pm4py.iterrows():
    timestamp = int(row["software:starttimenano"])
    date = datetime.datetime.fromtimestamp(timestamp / 1e9)
    log_pm4py.loc[index, "time:timestamp"] = date

log_pm4py

,software:classobject,software:class,software:callerclassobject,software:callerclass,software:linenumber,lifecycle:transition,software:callerpackage,software:threadid,software:starttimenano,software:parametertypeset,software:package,concept:name,software:endtimenano,software:callermethod,case:concept:name,software:returnvalue,Nested_Flag,software:parametervalueset,time:timestamp
0,1909173090,Console,@execution1,Console,37,start,com.verum.examples.AlarmSystem.Console,1,1780228355339781,,com.verum.examples.AlarmSystem.Console,init(),1780228355339781,main(),execution1,NaN,NaN,NaN,1970-01-21 17:30:28.355340
1,1585995280,LogWindow,872627444,Console,110,start,com.verum.examples.AlarmSystem.Console,15,1780229123870087,com.verum.examples.AlarmSystem.Console.Console...,com.verum.examples.AlarmSystem.Console,init(),1780229123870087,init(),execution1,NaN,NaN,NaN,1970-01-21 17:30:29.123870
2,0,AlarmSystemComponent,872627444,Console,114,start,com.verum.examples.AlarmSystem.Console,15,1780229125827882,,com.verum.examples.AlarmSystem,getInstance(),1780229740074283,init(),execution1,1574600022,0,,1970-01-21 17:30:29.125828
3,1585995280,LogWindow,872627444,Console,306,start,com.verum.examples.AlarmSystem.Console,15,1780229163020426,java.lang.String,com.verum.examples.AlarmSystem.Console,showInfo(),1780229177334824,traceHandler(),execution1,void,NaN,476977873,1970-01-21 17:30:29.163020
4,673733348,State,673733348,AlarmSystem,308,start,com.verum.examples,15,1780229406022635,com.verum.examples.AlarmSystem.AlarmSystemImpl...,com.verum.examples.AlarmSystem,init(),1780229406022635,AlarmSystemImplScope$NotActivated(),execution1,NaN,0,NaN,1970-01-21 17:30:29.406023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,1634888284,LogWindow,1751432579,Console,306,start,com.verum.examples.AlarmSystem.Console,19,1780414976124994,java.lang.String,com.verum.examples.AlarmSystem.Console,showInfo(),1780414977585856,traceHandler(),execution5,void,NaN,2127570078,1970-01-21 17:33:34.976125
931,0,Console,1921707273,Console,237,start,com.verum.examples.AlarmSystem.Console,15,1780414988771545,com.verum.examples.AlarmSystem.Console.Console,com.verum.examples.AlarmSystem.Console,access$3(),1780414989004189,run(),execution5,1906576862,NaN,1751432579,1970-01-21 17:33:34.988772
932,1751432579,Console,1921707273,Console,237,start,com.verum.examples.AlarmSystem.Console,15,1780414989127780,java.lang.String,com.verum.examples.AlarmSystem.Console,createImageIcon(),1780414989818867,run(),execution5,320094749,NaN,306384091,1970-01-21 17:33:34.989128
933,0,Console,1921707273,Console,238,start,com.verum.examples.AlarmSystem.Console,15,1780414990106678,com.verum.examples.AlarmSystem.Console.Console...,com.verum.examples.AlarmSystem.Console,access$4(),1780414990242244,run(),execution5,void,NaN,"1751432579,0",1970-01-21 17:33:34.990107


In [ ]:
import maxes.notebooks
import maxes.notebooks.utils
import pathlib

destination_file_path = os.path.join("software_data_analytics", "processed")
destination_file_path = maxes.notebooks.utils.get_data_path(destination_file_path)
pathlib.Path(destination_file_path).mkdir(exist_ok=True)

destination_file_path = os.path.join("software_data_analytics", "processed", "alarm_system.xes")
destination_file_path = maxes.notebooks.utils.get_data_path(destination_file_path)
destination_file_path

In [38]:
pm4py.write_xes(log_pm4py, destination_file_path)

exporting log, completed traces ::   0%|          | 0/5 [00:00<?, ?it/s]

In [26]:
import datetime
date1 = datetime.datetime.fromtimestamp(175174609198051 / 1e6)
print(date1)
date2 = datetime.datetime.fromtimestamp(175174636054676 / 1e6)
print(date2)

print(date2 - date1)

1975-07-21 14:36:49.198051
1975-07-21 14:37:16.054676
0:00:26.856625


In [40]:
log_pm4py_2 = pm4py.read_xes(destination_file_path)
log_pm4py_2

parsing log, completed traces ::   0%|          | 0/5 [00:00<?, ?it/s]

,software:classobject,software:class,software:callerclassobject,software:callerclass,software:linenumber,lifecycle:transition,software:callerpackage,software:threadid,software:starttimenano,software:parametertypeset,software:package,concept:name,software:endtimenano,software:callermethod,software:returnvalue,Nested_Flag,software:parametervalueset,time:timestamp,case:concept:name
0,1909173090,Console,@execution1,Console,37,start,com.verum.examples.AlarmSystem.Console,1,1780228355339781,,com.verum.examples.AlarmSystem.Console,init(),1780228355339781,main(),NaN,NaN,NaN,1970-01-21 17:30:28.355340+00:00,execution1
1,1585995280,LogWindow,872627444,Console,110,start,com.verum.examples.AlarmSystem.Console,15,1780229123870087,com.verum.examples.AlarmSystem.Console.Console...,com.verum.examples.AlarmSystem.Console,init(),1780229123870087,init(),NaN,NaN,NaN,1970-01-21 17:30:29.123870+00:00,execution1
2,0,AlarmSystemComponent,872627444,Console,114,start,com.verum.examples.AlarmSystem.Console,15,1780229125827882,,com.verum.examples.AlarmSystem,getInstance(),1780229740074283,init(),1574600022,0,,1970-01-21 17:30:29.125828+00:00,execution1
3,1585995280,LogWindow,872627444,Console,306,start,com.verum.examples.AlarmSystem.Console,15,1780229163020426,java.lang.String,com.verum.examples.AlarmSystem.Console,showInfo(),1780229177334824,traceHandler(),void,NaN,476977873,1970-01-21 17:30:29.163020+00:00,execution1
4,673733348,State,673733348,AlarmSystem,308,start,com.verum.examples,15,1780229406022635,com.verum.examples.AlarmSystem.AlarmSystemImpl...,com.verum.examples.AlarmSystem,init(),1780229406022635,AlarmSystemImplScope$NotActivated(),NaN,0,NaN,1970-01-21 17:30:29.406023+00:00,execution1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,1634888284,LogWindow,1751432579,Console,306,start,com.verum.examples.AlarmSystem.Console,19,1780414976124994,java.lang.String,com.verum.examples.AlarmSystem.Console,showInfo(),1780414977585856,traceHandler(),void,NaN,2127570078,1970-01-21 17:33:34.976125+00:00,execution5
931,0,Console,1921707273,Console,237,start,com.verum.examples.AlarmSystem.Console,15,1780414988771545,com.verum.examples.AlarmSystem.Console.Console,com.verum.examples.AlarmSystem.Console,access$3(),1780414989004189,run(),1906576862,NaN,1751432579,1970-01-21 17:33:34.988772+00:00,execution5
932,1751432579,Console,1921707273,Console,237,start,com.verum.examples.AlarmSystem.Console,15,1780414989127780,java.lang.String,com.verum.examples.AlarmSystem.Console,createImageIcon(),1780414989818867,run(),320094749,NaN,306384091,1970-01-21 17:33:34.989128+00:00,execution5
933,0,Console,1921707273,Console,238,start,com.verum.examples.AlarmSystem.Console,15,1780414990106678,com.verum.examples.AlarmSystem.Console.Console...,com.verum.examples.AlarmSystem.Console,access$4(),1780414990242244,run(),void,NaN,"1751432579,0",1970-01-21 17:33:34.990107+00:00,execution5


In [42]:
from maxes.xes_loader2 import XesLoader

loader = XesLoader()
log = loader.load(destination_file_path)
print(len(log.traces))

print(loader.errors)


5
['Event attribute software:returnvalue has multiple types: float: 216, string: 719', 'Event attribute Nested_Flag has multiple types: float: 747, string: 188', 'Event attribute software:parametervalueset has multiple types: float: 216, string: 719']


In [43]:
from maxes.generators.xes_generator.xes_generator3 import XesGenerator3

# Initialize generator
xes_generator = XesGenerator3(attributes_models=[], traces_count=1)
xes_generator.fit(log)

In [46]:
x = [len(trace.events) for trace in log.traces]
x

[58, 116, 216, 201, 344]

In [ ]:
xes_generator.generate()

In [57]:
xes_generator.trace_length_range_

(58, 344)

In [65]:
%autoreload
import maxes.graphs

maxes.graphs.random_weighted_traverse(
                        graph=xes_generator.sequence_graph_,
                        min_steps=xes_generator.trace_length_range_[0],
                        max_steps=xes_generator.trace_length_range_[1],
                        weight_feature_name="frequency",
                    )

whatever


[]

In [67]:
graph = xes_generator.sequence_graph_
first_node_choices = {node: data["first"]
                        for node, data in graph.nodes.items()
                        if data.get("first")}
first_node_choices

{('init()', 'start'): 5}

# bpic2020__request_for_payment

In [ ]:
file_path = maxes.data.load_files.get_path_bpic2020__request_for_payment()
file_path

In [69]:
log_pm4py = pm4py.read_xes(file_path)
log_pm4py

parsing log, completed traces ::   0%|          | 0/6886 [00:00<?, ?it/s]

,id,org:resource,concept:name,time:timestamp,org:role,case:Rfp_id,case:Project,case:Task,case:concept:name,case:OrganizationalEntity,case:Cost Type,case:RequestedAmount,case:Activity,case:RfpNumber
0,st_step 148220_0,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,2017-01-09 09:17:18+00:00,EMPLOYEE,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215
1,st_step 148221_0,STAFF MEMBER,Request For Payment FINAL_APPROVED by SUPERVISOR,2017-01-09 09:18:00+00:00,SUPERVISOR,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215
2,st_step 148222_0,STAFF MEMBER,Request For Payment REJECTED by MISSING,2017-01-10 12:42:32+00:00,MISSING,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215
3,st_step 148219_0,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,2017-03-03 09:51:13+00:00,EMPLOYEE,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215
4,st_step 148218_0,STAFF MEMBER,Request For Payment APPROVED by PRE_APPROVER,2017-03-03 09:51:42+00:00,PRE_APPROVER,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36791,st_step 185004_0,STAFF MEMBER,Request For Payment APPROVED by ADMINISTRATION,2018-12-29 12:35:02+00:00,ADMINISTRATION,request for payment 185000,project 147860,task 152704,request for payment 185000,organizational unit 65468,0,15.409660,activity 505,request for payment number 185001
36792,st_step 185003_0,STAFF MEMBER,Request For Payment APPROVED by BUDGET OWNER,2019-01-03 09:27:20+00:00,BUDGET OWNER,request for payment 185000,project 147860,task 152704,request for payment 185000,organizational unit 65468,0,15.409660,activity 505,request for payment number 185001
36793,st_step 185005_0,STAFF MEMBER,Request For Payment FINAL_APPROVED by SUPERVISOR,2019-01-08 09:00:39+00:00,SUPERVISOR,request for payment 185000,project 147860,task 152704,request for payment 185000,organizational unit 65468,0,15.409660,activity 505,request for payment number 185001
36794,rp_request for payment 185000_15,SYSTEM,Request Payment,2019-01-08 09:29:14+00:00,UNDEFINED,request for payment 185000,project 147860,task 152704,request for payment 185000,organizational unit 65468,0,15.409660,activity 505,request for payment number 185001


In [73]:
# Preprocessing

log_pm4py["lifecycle:transition"] = "start"

import maxes.notebooks
import maxes.notebooks.utils
import pathlib

destination_file_path = os.path.join("bpic2020", "processed")
destination_file_path = maxes.notebooks.utils.get_data_path(destination_file_path)
pathlib.Path(destination_file_path).mkdir(exist_ok=True)

destination_file_path = os.path.join("bpic2020", "processed", "request_for_payment.xes")
destination_file_path = maxes.notebooks.utils.get_data_path(destination_file_path)
destination_file_path

pm4py.write_xes(log_pm4py, destination_file_path)

exporting log, completed traces ::   0%|          | 0/6886 [00:00<?, ?it/s]

In [ ]:
file_path = destination_file_path
file_path

In [95]:
log_pm4py = pm4py.read_xes(file_path)
log_pm4py

parsing log, completed traces ::   0%|          | 0/6886 [00:00<?, ?it/s]

,id,org:resource,concept:name,time:timestamp,org:role,lifecycle:transition,case:Rfp_id,case:Project,case:Task,case:concept:name,case:OrganizationalEntity,case:Cost Type,case:RequestedAmount,case:Activity,case:RfpNumber
0,st_step 148220_0,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,2017-01-09 09:17:18+00:00,EMPLOYEE,start,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215
1,st_step 148221_0,STAFF MEMBER,Request For Payment FINAL_APPROVED by SUPERVISOR,2017-01-09 09:18:00+00:00,SUPERVISOR,start,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215
2,st_step 148222_0,STAFF MEMBER,Request For Payment REJECTED by MISSING,2017-01-10 12:42:32+00:00,MISSING,start,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215
3,st_step 148219_0,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,2017-03-03 09:51:13+00:00,EMPLOYEE,start,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215
4,st_step 148218_0,STAFF MEMBER,Request For Payment APPROVED by PRE_APPROVER,2017-03-03 09:51:42+00:00,PRE_APPROVER,start,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36791,st_step 185004_0,STAFF MEMBER,Request For Payment APPROVED by ADMINISTRATION,2018-12-29 12:35:02+00:00,ADMINISTRATION,start,request for payment 185000,project 147860,task 152704,request for payment 185000,organizational unit 65468,0,15.409660,activity 505,request for payment number 185001
36792,st_step 185003_0,STAFF MEMBER,Request For Payment APPROVED by BUDGET OWNER,2019-01-03 09:27:20+00:00,BUDGET OWNER,start,request for payment 185000,project 147860,task 152704,request for payment 185000,organizational unit 65468,0,15.409660,activity 505,request for payment number 185001
36793,st_step 185005_0,STAFF MEMBER,Request For Payment FINAL_APPROVED by SUPERVISOR,2019-01-08 09:00:39+00:00,SUPERVISOR,start,request for payment 185000,project 147860,task 152704,request for payment 185000,organizational unit 65468,0,15.409660,activity 505,request for payment number 185001
36794,rp_request for payment 185000_15,SYSTEM,Request Payment,2019-01-08 09:29:14+00:00,UNDEFINED,start,request for payment 185000,project 147860,task 152704,request for payment 185000,organizational unit 65468,0,15.409660,activity 505,request for payment number 185001


In [96]:
log = XesLoader().load(file_path)
log.update_df()

,id,org:resource,concept:name,time:timestamp,org:role,lifecycle:transition,case:Rfp_id,case:Project,case:Task,case:concept:name,case:OrganizationalEntity,case:Cost Type,case:RequestedAmount,case:Activity,case:RfpNumber
0,st_step 148220_0,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,2017-01-09 09:17:18+00:00,EMPLOYEE,start,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215
1,st_step 148221_0,STAFF MEMBER,Request For Payment FINAL_APPROVED by SUPERVISOR,2017-01-09 09:18:00+00:00,SUPERVISOR,start,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215
2,st_step 148222_0,STAFF MEMBER,Request For Payment REJECTED by MISSING,2017-01-10 12:42:32+00:00,MISSING,start,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215
3,st_step 148219_0,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,2017-03-03 09:51:13+00:00,EMPLOYEE,start,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215
4,st_step 148218_0,STAFF MEMBER,Request For Payment APPROVED by PRE_APPROVER,2017-03-03 09:51:42+00:00,PRE_APPROVER,start,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,st_step 185004_0,STAFF MEMBER,Request For Payment APPROVED by ADMINISTRATION,2018-12-29 12:35:02+00:00,ADMINISTRATION,start,request for payment 185000,project 147860,task 152704,request for payment 185000,organizational unit 65468,0,15.409660,activity 505,request for payment number 185001
2,st_step 185003_0,STAFF MEMBER,Request For Payment APPROVED by BUDGET OWNER,2019-01-03 09:27:20+00:00,BUDGET OWNER,start,request for payment 185000,project 147860,task 152704,request for payment 185000,organizational unit 65468,0,15.409660,activity 505,request for payment number 185001
3,st_step 185005_0,STAFF MEMBER,Request For Payment FINAL_APPROVED by SUPERVISOR,2019-01-08 09:00:39+00:00,SUPERVISOR,start,request for payment 185000,project 147860,task 152704,request for payment 185000,organizational unit 65468,0,15.409660,activity 505,request for payment number 185001
4,rp_request for payment 185000_15,SYSTEM,Request Payment,2019-01-08 09:29:14+00:00,UNDEFINED,start,request for payment 185000,project 147860,task 152704,request for payment 185000,organizational unit 65468,0,15.409660,activity 505,request for payment number 185001


In [97]:
xes_generator = XesGenerator3()
xes_generator.fit(log)

In [98]:
log_pm4py["case:concept:name"].nunique()

6886

In [103]:
xes_generator.traces_count = 10000
generated_log = xes_generator.generate()
print(len(generated_log.traces))

10000


In [104]:
for trace_index, trace in enumerate(generated_log.traces):
    trace.attributes["concept:name"] = str(trace_index)

generated_log.update_df(with_trace_attributes=True)
generated_log_pm4py = generated_log.df
generated_log_pm4py

,concept:name,lifecycle:transition,time:timestamp,case:concept:name
0,Request For Payment SUBMITTED by EMPLOYEE,start,2018-04-12 13:34:25.327098+00:00,0
1,Request For Payment REJECTED by ADMINISTRATION,start,2018-04-12 13:34:55.587563+00:00,0
0,Request For Payment SUBMITTED by EMPLOYEE,start,2018-04-25 15:00:10.920074+00:00,1
1,Request For Payment FINAL_APPROVED by SUPERVISOR,start,2018-04-25 15:01:17.691072+00:00,1
0,Request For Payment SUBMITTED by EMPLOYEE,start,2018-01-08 18:44:31.801217+00:00,2
...,...,...,...,...
1,Request For Payment APPROVED by ADMINISTRATION,start,2018-07-04 15:11:15.400349+00:00,9998
2,Request For Payment FINAL_APPROVED by SUPERVISOR,start,2018-07-04 17:08:42.068387+00:00,9998
0,Request For Payment SUBMITTED by EMPLOYEE,start,2018-07-03 15:27:24.155138+00:00,9999
1,Request For Payment APPROVED by ADMINISTRATION,start,2018-07-03 15:27:37.324676+00:00,9999


In [105]:
net, im, fm = pm4py.discover_petri_net_inductive(log_pm4py)


In [106]:
print("Precision: ", pm4py.conformance.precision_token_based_replay(generated_log_pm4py, net, im, fm))


replaying log with TBR, completed traces ::   0%|          | 0/16 [00:00<?, ?it/s]

Precision:  0.419220266019661


In [ ]:
0.3749484010081353

In [ ]:
# bpic2020
# photo copier
# alarm system
# ccc19
# activities
# simple

# Simple.xes

In [ ]:
file_path = "/vt/md/maxes/maxes/data/software_data_analytics/processed/alarm_system.xes"
file_path

In [121]:
log_pm4py = pm4py.read_xes(file_path)
log_pm4py

parsing log, completed traces ::   0%|          | 0/5 [00:00<?, ?it/s]

,software:classobject,software:class,software:callerclassobject,software:callerclass,software:linenumber,lifecycle:transition,software:callerpackage,software:threadid,software:starttimenano,software:parametertypeset,software:package,concept:name,software:endtimenano,software:callermethod,software:returnvalue,Nested_Flag,software:parametervalueset,time:timestamp,case:concept:name
0,1909173090,Console,@execution1,Console,37,start,com.verum.examples.AlarmSystem.Console,1,1780228355339781,,com.verum.examples.AlarmSystem.Console,init(),1780228355339781,main(),NaN,NaN,NaN,1970-01-21 17:30:28.355340+00:00,execution1
1,1585995280,LogWindow,872627444,Console,110,start,com.verum.examples.AlarmSystem.Console,15,1780229123870087,com.verum.examples.AlarmSystem.Console.Console...,com.verum.examples.AlarmSystem.Console,init(),1780229123870087,init(),NaN,NaN,NaN,1970-01-21 17:30:29.123870+00:00,execution1
2,0,AlarmSystemComponent,872627444,Console,114,start,com.verum.examples.AlarmSystem.Console,15,1780229125827882,,com.verum.examples.AlarmSystem,getInstance(),1780229740074283,init(),1574600022,0,,1970-01-21 17:30:29.125828+00:00,execution1
3,1585995280,LogWindow,872627444,Console,306,start,com.verum.examples.AlarmSystem.Console,15,1780229163020426,java.lang.String,com.verum.examples.AlarmSystem.Console,showInfo(),1780229177334824,traceHandler(),void,NaN,476977873,1970-01-21 17:30:29.163020+00:00,execution1
4,673733348,State,673733348,AlarmSystem,308,start,com.verum.examples,15,1780229406022635,com.verum.examples.AlarmSystem.AlarmSystemImpl...,com.verum.examples.AlarmSystem,init(),1780229406022635,AlarmSystemImplScope$NotActivated(),NaN,0,NaN,1970-01-21 17:30:29.406023+00:00,execution1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,1634888284,LogWindow,1751432579,Console,306,start,com.verum.examples.AlarmSystem.Console,19,1780414976124994,java.lang.String,com.verum.examples.AlarmSystem.Console,showInfo(),1780414977585856,traceHandler(),void,NaN,2127570078,1970-01-21 17:33:34.976125+00:00,execution5
931,0,Console,1921707273,Console,237,start,com.verum.examples.AlarmSystem.Console,15,1780414988771545,com.verum.examples.AlarmSystem.Console.Console,com.verum.examples.AlarmSystem.Console,access$3(),1780414989004189,run(),1906576862,NaN,1751432579,1970-01-21 17:33:34.988772+00:00,execution5
932,1751432579,Console,1921707273,Console,237,start,com.verum.examples.AlarmSystem.Console,15,1780414989127780,java.lang.String,com.verum.examples.AlarmSystem.Console,createImageIcon(),1780414989818867,run(),320094749,NaN,306384091,1970-01-21 17:33:34.989128+00:00,execution5
933,0,Console,1921707273,Console,238,start,com.verum.examples.AlarmSystem.Console,15,1780414990106678,com.verum.examples.AlarmSystem.Console.Console...,com.verum.examples.AlarmSystem.Console,access$4(),1780414990242244,run(),void,NaN,"1751432579,0",1970-01-21 17:33:34.990107+00:00,execution5


In [122]:
log_pm4py["case:concept:name"].nunique()

5

In [123]:
# Define "folding" algorithm

def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i+n]

traces_concept_names = log_pm4py['case:concept:name'].unique()
rng = np.random.default_rng(123)
rng.shuffle(traces_concept_names)

# Folds of size 5
list(chunks(traces_concept_names, 5))

[array(['execution5', 'execution1', 'execution3', 'execution4',
        'execution2'], dtype=object)]

In [124]:
from maxes.generators.xes_generator.xes_generator3 import XesGenerator3
from maxes.xes_loader2 import XesLog

from pm4py.algo.evaluation.replay_fitness.variants import token_replay as fitness_tbr
from pm4py.algo.evaluation.generalization.variants import token_based as generalization_tbr
from pm4py.algo.evaluation.precision.variants import etconformance_token as precision_tbr

# Full
# fold_sizes = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048]
# fold_size_attempts_count = 10
# fold_generation_attempts_count = 10

# Debug
fold_sizes = [1, 2, 3, 4, 5]
fold_sizes = [1]
fold_size_attempts_count = 1
fold_generation_attempts_count = 1

results = []

for fold_size in fold_sizes:
    print(f"{fold_size=}")

    for fold_size_attempt in range(fold_size_attempts_count):
        shuffled_traces_concept_names = list(traces_concept_names)
        rng.shuffle(shuffled_traces_concept_names)

        folds = chunks(traces_concept_names, fold_size)

        for fold_index, fold in enumerate(folds):
            filtered_log_df = log_pm4py[log_pm4py['case:concept:name'].isin(fold)]

            print(f"{fold_size=}, {fold_size_attempt=}, {fold_index=}")

            # load log
            log = XesLog()
            log.df = filtered_log_df
            log.update_traces_from_df()

            # Initialize generator
            xes_generator = XesGenerator3(attributes_models=[])
            xes_generator.fit(log)

            for fold_generation_attempt in range(fold_generation_attempts_count):
                generated_log = xes_generator.generate()
                for trace_index, trace in enumerate(generated_log.traces):
                    trace.attributes["concept:name"] = str(trace_index)

                generated_log.update_df(with_trace_attributes=True)
                generated_log_pm4py = generated_log.df

                # evaluate

                fitness = fitness_tbr.apply(generated_log_pm4py, net, im, fm, {'show_progress_bar': False})
                generalization = generalization_tbr.apply(generated_log_pm4py, net, im, fm, {'show_progress_bar': False})
                precision = precision_tbr.apply(generated_log_pm4py, net, im, fm, {'show_progress_bar': False})

                results.append({
                    'fold_size': fold_size,
                    'fold_size_attempt': fold_size_attempt,
                    'fold_index': fold_index,
                    'fold_generation_attempt': fold_generation_attempt,
                    'perc_fit_traces': fitness['perc_fit_traces'],
                    'average_trace_fitness': fitness['average_trace_fitness'],
                    'log_fitness': fitness['log_fitness'],
                    'percentage_of_fitting_traces': fitness['percentage_of_fitting_traces'],
                    'generalization': generalization,
                    'precision': precision
                })

fold_size=1
fold_size=1, fold_size_attempt=0, fold_index=0


KeyboardInterrupt: 

In [125]:
# precision = precision_tbr.apply(generated_log_pm4py, net, im, fm, {'show_progress_bar': False})

pm4py.conformance.precision_token_based_replay(generated_log_pm4py, net, im, fm)

replaying log with TBR, completed traces ::   0%|          | 0/343 [00:00<?, ?it/s]

0.0

In [ ]:
import maxes.data.load_files
from maxes.analyze_xes import AnalyzeXes

file_path = maxes.data.load_files.get_path_photo_copier()
file_path

AnalyzeXes().analyze(file_path)

In [ ]:
import maxes.data.load_files
from maxes.analyze_xes import AnalyzeXes

file_path = maxes.data.load_files.get_path_software_data_analytics__alarm_system()
file_path

AnalyzeXes().analyze(file_path)

In [5]:
import maxes.data.load_files
from maxes.analyze_xes import AnalyzeXes

file_path = "../data/other/simple.xes"
file_path

AnalyzeXes().analyze(file_path)

{'File': {'Path': '../data/other/simple.xes',
  'Size in bytes': 13826901,
  'Size human-readable': '13.2 MiB'},
 'XML': {'XES 2014': {'Passed': False,
   'Error': {'Message': "failed validating <Element '{http://www.xes-standard.org/}log' at 0x148ed92b0> with XMLSchema10(name='xes2014-original.xsd', namespace='http://www.xes-standard.org')",
    'Reason': "<Element '{http://www.xes-standard.org/}log' at 0x148ed92b0> is not an element of the schema"},
   'Time': 0.28564977645874023},
  'XES 2023': {'Passed': False,
   'Error': {'Message': "failed validating <Element '{http://www.xes-standard.org/}log' at 0x337a829d0> with XMLSchema10(name='xes2023-original.xsd', namespace='http://www.xes-standard.org')",
    'Reason': "<Element '{http://www.xes-standard.org/}log' at 0x337a829d0> is not an element of the schema"},
   'Time': 0.29230785369873047},
  'XES 2023 (Extended)': {'Passed': False,
   'Error': {'Message': "failed validating <Element '{http://www.xes-standard.org/}log' at 0x337776